In [1]:
from dotenv import load_dotenv
from portia.cli import CLIExecutionHooks
from portia import *
import requests
import xml.etree.ElementTree as ET
from pydantic import BaseModel, Field
from typing import Generic, TypeVar, List

from my_custom_tools.arXivTool import ArXivTool
from my_custom_tools.DownloadTool import DownloadPaperTool
from my_custom_tools.PDFReaderTool import PDFReaderTool

load_dotenv(override=True)

DownloadPaperSchema initialized
papers: annotation=NoneType required=True description="A list of dicts, each with 'title', 'link', and 'summary'."


True

In [2]:
from my_custom_tools.registry import custom_tool_registry

my_config = Config.from_default()
complete_tool_registry = PortiaToolRegistry(my_config) + custom_tool_registry

portia = Portia(config = my_config,
                tools = complete_tool_registry,
                execution_hooks=CLIExecutionHooks(),)

In [8]:
constraints = []
# topic = input("What topic are you interested in covering today?")
# number_of_papers = int(input("How many papers do you want to download?"))
topic = "Using LLMs for Interior Design"
number_of_papers = 1

# task = (
#     lambda: f"""Please find and download {number_of_papers} paper{'s' * (number_of_papers > 1)}
#             on the topic of {topic} using the ArXivTool. Then run the PDFReaderTool and 
#             summarize the content of the PDF files returned by the tool."""
# )
task = (
            lambda : f"""You are a research assistant running these tasks: 
                      - Find and download {number_of_papers} paper{'s' * (number_of_papers > 1)} on the topic of {topic} using the ArXivTool. 
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - From the full text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.
                      
                        Take into account these constraints: {constraints}
                      """
        )
# task = (
#     lambda: f"""Please find and download {number_of_papers} paper{'s' * (number_of_papers > 1)}
#             on the topic of {topic} using the ArXivTool."""
# )

plan = portia.plan(task())
print("\nHere are the steps in the generated plan:")
[print(step.model_dump_json(indent=2)) for step in plan.steps]

2025-04-11 23:03:11.557 | INFO | portia.portia:plan:197 - Running planning_agent for query - You are a research assistant running these tasks: 
                      - Find and download 1 paper on the topic of Using LLMs for Interior Design using the ArXivTool. 
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - From the full text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.

                        Take into account these constraints: []
                      
2025-04-11 23:03:57.525 | INFO | portia.portia:plan:222 - Plan created with 4 steps | {'plan': 'plan-e778

[None, None, None, None]

In [9]:
# Iterate on the plan with the user until they are happy with it
ready_to_proceed = False
while not ready_to_proceed:
    user_input = input("Are you happy with the plan? (y/n):\n")
    if user_input == "y":
        ready_to_proceed = True
    else:
        user_input = input("Any additional guidance for the planner?:\n")
        constraints.append(user_input)
        plan = portia.plan(task())
        print("\nHere are the updated steps in the plan:")
        [print(step.model_dump_json(indent=2)) for step in plan.steps]

# Execute the plan
print("\nThe plan will now be executed. Please wait...")
plan_run = portia.run_plan(plan)

2025-04-11 23:04:45.975 | INFO | portia.portia:plan:197 - Running planning_agent for query - You are a research assistant running these tasks: 
                      - Find and download 1 paper on the topic of Using LLMs for Interior Design using the ArXivTool. 
                      - Run the PDFReaderTool to extract the full text from the pdfs in the local folder.
                      - From the full text, extract the core mathematical and scientific concepts required 
                        to understand the paper. Focus only on generalizable topics that could be included 
                        in a learning pathway or curriculum—avoid content specific to the study's location, 
                        data, or outcomes. List only the overarching topics, with no explanations or extra text.

                        Take into account these constraints: ['I want to stop after downloading, and not proceed with the reading and summarising and topics. ']
                      
2025-04-